In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open("names.txt").read().splitlines()
print(words[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [3]:
len(words)

32033

In [4]:
chs = sorted(list(set("".join(words))))
stoi = {s: i + 1 for i, s in enumerate(chs)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# building the dataset
block_size = 3
X, Y = [], []

for w in words[:5]:
    print(w)
    ctx = [stoi["."]] * block_size
    for c in w + ".":
        X.append(ctx)
        Y.append(stoi[c])
        print("".join(itos[i] for i in ctx), "-->", c)
        ctx = ctx[1:] + [stoi[c]]

X, Y = torch.tensor(X), torch.tensor(Y)

emma
... --> e
..e --> m
.em --> m
emm --> a
mma --> .
olivia
... --> o
..o --> l
.ol --> i
oli --> v
liv --> i
ivi --> a
via --> .
ava
... --> a
..a --> v
.av --> a
ava --> .
isabella
... --> i
..i --> s
.is --> a
isa --> b
sab --> e
abe --> l
bel --> l
ell --> a
lla --> .
sophia
... --> s
..s --> o
.so --> p
sop --> h
oph --> i
phi --> a
hia --> .


In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [7]:
C = torch.randn((27, 2))

In [8]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [9]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [16]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [17]:
h

tensor([[ 0.0189, -0.5303, -0.9726,  ...,  0.7094, -0.5848, -0.6977],
        [-0.2863, -0.9320, -0.9313,  ...,  0.9705, -0.8888, -0.3050],
        [ 0.3816,  0.9491, -0.9913,  ..., -0.6031,  0.5563, -0.2103],
        ...,
        [-1.0000, -0.7560,  0.9598,  ...,  0.7605,  0.4052,  0.8948],
        [-0.9991,  0.0723, -0.2244,  ...,  0.8490,  0.9502,  0.8520],
        [-0.8150, -0.9990, -0.9999,  ..., -0.3899, -0.3219, -0.1929]])

In [18]:
h.shape

torch.Size([32, 100])

In [19]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [20]:
logits = h @ W2 + b2

In [21]:
logits.shape

torch.Size([32, 27])

In [22]:
counts = logits.exp()

In [23]:
prob = counts / counts.sum(1, True)

In [24]:
prob.shape

torch.Size([32, 27])

In [25]:
prob[0].sum()

tensor(1.0000)

In [28]:
loss = -prob[:, Y].log().mean()
loss

tensor(15.5067)